In [ ]:
!pip install --upgrade -q accelerate peft bitsandbytes transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, pipeline, TextStreamer)

In [ ]:
llama_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = 'unsloth/Meta-Llama-3.1-8B',
                                                   quantization_config = BitsAndBytesConfig(load_in_8bit = True,
                                                                                            max_seq_length = 2048,
                                                                      bnb_4bit_compute_dtype = getattr(torch, 'float16')))
llama_model.config.use_cache = False

Unused kwargs: ['max_seq_length']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/926 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

In [ ]:
llama_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path = 'unsloth/Meta-Llama-3.1-8B',
                                                max_seq_length = 2048,
                                                trust_remote_code = True)

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [ ]:
hausa_format = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}


###Output:
{}"""

EOS_Token = llama_tokenizer.eos_token

def formatting(dataset):
  instructions = dataset['instruction']
  inputs = dataset['input']
  outputs = dataset['output']
  texts = []
  for instruction , input, output in zip(instructions, inputs, outputs):
    text = hausa_format.format(instruction, input, output) + EOS_Token
    texts.append(text)
  return {'text':texts}

pass
dataset = load_dataset(path = 'saillab/alpaca_hausa_taco', split = 'train')
dataset = dataset.map(formatting, batched = True)

README.md:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/89.5M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/22.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/49601 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12401 [00:00<?, ? examples/s]

Map:   0%|          | 0/49601 [00:00<?, ? examples/s]

In [ ]:
trainer = SFTTrainer(
    model = llama_model,
    tokenizer = llama_tokenizer,
    train_dataset = dataset,
    peft_config = LoraConfig(
       r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_rslora = False,
    loftq_config = None,
    ),
    dataset_text_field = "text",
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        max_steps = 29,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_bnb_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        fp16=True,
        gradient_checkpointing=True,
    )
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, dataset_num_proc. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:314: UserWarning: You passed a `dataset_num_proc` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`

Map (num_proc=2):   0%|          | 0/49601 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,1.514600
2,1.554900
3,1.824500
4,1.448000
5,1.543300
6,1.577400
7,1.523400
8,1.169300
9,1.140700
10,1.435900


TrainOutput(global_step=29, training_loss=1.2905905883887718, metrics={'train_runtime': 251.9681, 'train_samples_per_second': 0.23, 'train_steps_per_second': 0.115, 'total_flos': 1211498673807360.0, 'train_loss': 1.2905905883887718, 'epoch': 0.001169331263482591})

In [ ]:
!pip install fastapi uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.8 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok

In [ ]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import LlamaForCausalLM, PreTrainedTokenizerFast

nest_asyncio.apply()

app = FastAPI()

class InputData(BaseModel):
    input_text: str

@app.post("/predict")
async def generate_text(data: InputData):
    try:
        # Check if the input text is in Hausa (you can adjust the condition based on your criteria)
        if is_hausa(data.input_text):
            instruction = "Translate to the user in English."
        else:
            instruction = "Chat to the user in Hausa."

        # Format the input to include the instruction
        formatted_input = hausa_format.format(instruction, data.input_text, "")

        inputs = llama_tokenizer(formatted_input, return_tensors="pt").to("cuda")

        outputs = llama_model.generate(**inputs, max_length=512)
        output_text = llama_tokenizer.decode(outputs[0], skip_special_tokens=True)

        return {"generated_text": output_text}

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

def is_hausa(text: str) -> bool:
    hausa_keywords = ["na", "ka", "ki", "mu", "su", "ni", "a", "in", "da"]
    return any(word in text for word in hausa_keywords)

from pyngrok import ngrok

ngrok.set_auth_token("2lF5wXiCnwiL7Gw02aZYf670KtN_6cgH5KX53byFqAiJbH7e6")
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


Public URL: NgrokTunnel: "https://ee45-34-125-65-94.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [405]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     105.112.26.16:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     105.112.26.16:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     105.112.26.16:0 - "POST /predict HTTP/1.1" 200 OK


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-10' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/lib/python3.10/asyncio/tasks.py", 

INFO:     105.112.26.16:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     105.112.26.16:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     105.112.26.16:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     105.112.26.16:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     105.112.26.16:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     105.112.26.16:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     105.112.26.16:0 - "GET /openapi.json HTTP/1.1" 200 OK
